In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from utils import read_pickle
from tqdm.notebook import tqdm

In [2]:
path = '/Users/diegofiori/Desktop/epfl/master_thesis/results/'
sim_code = 'cs'
density = read_pickle(path+f'physical_features_end_{sim_code}.pickle')
slices = read_pickle(path+f'slices_top_features_end_{sim_code}.pickle')

In [3]:
density = np.concatenate(density)

In [4]:
density.shape

(1888,)

In [5]:
slices.shape

(4, 7680, 11134)

In [6]:
df = pd.DataFrame(data=slices[0])

In [7]:
df.shape

(7680, 11134)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11124,11125,11126,11127,11128,11129,11130,11131,11132,11133
0,2.931996,4.086118,0.999678,0.908342,1.813822,2.293534,1.003000,0.915540,0.499631,0.412712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.935314,4.081612,0.999665,0.908276,1.823356,2.287220,1.003123,0.915417,0.499618,0.412651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.921623,4.101256,0.999652,0.908207,1.818434,2.302971,1.003139,0.915526,0.499605,0.412606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.943066,4.110412,0.999640,0.908151,1.830123,2.324676,1.003135,0.915781,0.499592,0.412542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.955779,4.089593,0.999627,0.907780,1.838863,2.304073,1.003237,0.915188,0.499580,0.412221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from pipeline_builder import get_pipeline_index

In [10]:
dict_transf = get_pipeline_index()
dict_transf

{'persistence_entropy': I[0, 2),
 'amplitude_bottleneck_inf': I[2, 4),
 'amplitude_wasserstein_1': I[4, 6),
 'amplitude_wasserstein_2': I[6, 8),
 'amplitude_landscape_1_1_100': I[8, 10),
 'amplitude_landscape_1_2_100': I[10, 12),
 'amplitude_landscape_2_1_100': I[12, 14),
 'amplitude_landscape_2_2_100': I[14, 16),
 'amplitude_betti_1_100': I[16, 18),
 'amplitude_betti_2_100': I[18, 20),
 'amplitude_heat_1_1.6_100': I[20, 22),
 'amplitude_heat_1_3.2_100': I[22, 24),
 'amplitude_heat_2_1.6_100': I[24, 26),
 'amplitude_heat_2_3.2_100': I[26, 28),
 'derivative_bottleneck_inf': I[28, 30),
 'derivative_wasserstein_1': I[30, 32),
 'derivative_wasserstein_2': I[32, 34),
 'heat_kernel_1.6': I[34, 5034),
 'heat_kernel_3.2': I[5034, 10034),
 'betti_curve': I[10034, 10134),
 'pers_landscape': I[10134, 11134)}

## Selecting features based on RandomForest

We firstly defined the regression problem.

In [11]:
import h5py

In [12]:
from scipy.io import loadmat
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code}.mat')
y = x['k_quantities']

In [13]:
y = np.concatenate([y[i, :, :] for i in range(y.shape[0])])

In [14]:
y.shape

(8000, 2)

In [15]:
y = y[:len(df)]

In [16]:
non_zero_ind = np.any(y>0, axis=1)

In [17]:
y1 = y[non_zero_ind, 0]
y2 = y[non_zero_ind, 1]

In [18]:
X = df.values[non_zero_ind]

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [20]:
from scipy.stats import pearsonr
def compute_correlation_with_old_features(target_feat, old_features_idx, X):
    corr = [pearsonr(target_feat, X[:, old_idx])[0] for old_idx in old_features_idx]
    return np.abs(corr)

In [21]:
# remove some features which are higly correlated
X = np.nan_to_num(X)
columns_idx = []
for i in tqdm(range(X.shape[1])):  
    feat_mat = X[:, i]
    if columns_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, columns_idx, X)]
        if np.max(corrs) < 0.95:
            columns_idx.append(i)
    else:
        columns_idx.append(i)
X = X[:, np.array(columns_idx)]

/Users/diegofiori/anaconda3/envs/giotto_env/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/diegofiori/anaconda3/envs/giotto_env/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [23]:
X.shape

(7680, 1574)

In [24]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2)

In [25]:
model1 = RandomForestRegressor(n_estimators=200)
model1.fit(X_train, y1_train)
model2 = RandomForestRegressor(n_estimators=200)
model2.fit(X_train, y2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [26]:
score_1 = model1.score(X_test, y1_test)
score_2 = model2.score(X_test, y2_test)
total_score = score_1 + score_2

In [27]:
score_2

0.8976069957477941

In [28]:
score_1

0.9338495962220583

In [29]:
total_score

1.8314565919698524

In [30]:
model1.predict(X_test[19:20])

array([0.02939604])

In [31]:
y1_test[19]

0.02826588956735257

In [32]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y1_test, model1.predict(X_test))

0.000569303432359422

In [33]:
print('relative error')
mean_absolute_error(y1_test, model1.predict(X_test))/np.mean(y1_test)

relative error


0.018493762205145182

In [34]:
feature_importance = score_1/total_score*model1.feature_importances_ + score_2/total_score*model2.feature_importances_


In [35]:
# group the features (we want to keep all the spatial dimensions)
n_features = slices.shape[2]
#feature_importance_temp = [feature_importance[i:i+n_features] for i in range(0, len(feature_importance), n_features)]
#feature_importance_temp = [np.mean(feature_importance_temp[i:i+80], axis=0) 
#                           for i in range(0, len(feature_importance_temp), 80)]

In [36]:
feature_importance_idx = [j for j in range(n_features)]
feature_importance_idx = [feature_importance_idx[i] for i in range(len(feature_importance_idx)) if i in columns_idx]
feature_importance_idx_sort = sorted(feature_importance_idx, reverse=True,
                                key=lambda x: feature_importance[feature_importance_idx.index(x)])

In [37]:
important_features_idx = []
threshold = 0.85
minimum_value = np.max(feature_importance)/100
for i, idx in enumerate(tqdm(feature_importance_idx_sort)):
    feat_value = feature_importance[feature_importance_idx.index(idx)]
    if feat_value < minimum_value:
        break
    feat_mat = X[:, feature_importance_idx.index(idx)]
    if important_features_idx:
        corrs = [compute_correlation_with_old_features(feat_mat, important_features_idx, X)]
        if np.max(corrs) < threshold:
            important_features_idx.append(i)
    else:
        important_features_idx.append(i)
important_features = [feature_importance_idx_sort[i] for i in important_features_idx]

In [38]:
from utils import write_pickle
write_pickle(path=path+f'selected_index_{sim_code}_single_field.pickle', array=important_features)

In [39]:
len(important_features)

73

In [41]:
important_features = [(0, i) for i in important_features]

In [42]:
important_features

[(0, 0),
 (0, 6),
 (0, 10185),
 (0, 10638),
 (0, 10717),
 (0, 10639),
 (0, 10236),
 (0, 10838),
 (0, 10186),
 (0, 10637),
 (0, 10689),
 (0, 10688),
 (0, 10690),
 (0, 4384),
 (0, 10788),
 (0, 2384),
 (0, 10240),
 (0, 10742),
 (0, 4385),
 (0, 11037),
 (0, 10695),
 (0, 10692),
 (0, 10739),
 (0, 10192),
 (0, 10738),
 (0, 10693),
 (0, 10745),
 (0, 10892),
 (0, 3161),
 (0, 4286),
 (0, 4238),
 (0, 10189),
 (0, 10743),
 (0, 10235),
 (0, 2034),
 (0, 10191),
 (0, 10988),
 (0, 10888),
 (0, 10842),
 (0, 10687),
 (0, 10697),
 (0, 10636),
 (0, 10661),
 (0, 10691),
 (0, 10737),
 (0, 10286),
 (0, 10187),
 (0, 10792),
 (0, 10188),
 (0, 3211),
 (0, 10701),
 (0, 10741),
 (0, 10190),
 (0, 10699),
 (0, 10696),
 (0, 10194),
 (0, 2184),
 (0, 10694),
 (0, 4289),
 (0, 4734),
 (0, 6393),
 (0, 10675),
 (0, 10238),
 (0, 10241),
 (0, 10700),
 (0, 10239),
 (0, 10193),
 (0, 10698),
 (0, 10635),
 (0, 10740),
 (0, 10744),
 (0, 10196),
 (0, 11087)]

In [43]:
important_features_all = read_pickle(path+'selected_index_cs.pickle')

In [45]:
intersection = set(important_features_all).intersection(set(important_features))

In [46]:
intersection

{(0, 10185),
 (0, 10186),
 (0, 10236),
 (0, 10637),
 (0, 10638),
 (0, 10639),
 (0, 10689),
 (0, 10691),
 (0, 10692),
 (0, 10699),
 (0, 10717),
 (0, 10838),
 (0, 10842)}

In [47]:
intersection_1 = set(important_features_all).intersection(set([(1, x[1]) for x in important_features]))
intersection_1

{(1, 6),
 (1, 10185),
 (1, 10186),
 (1, 10188),
 (1, 10235),
 (1, 10236),
 (1, 10637),
 (1, 10687),
 (1, 10688),
 (1, 10737),
 (1, 10740)}

In [48]:
intersection_2 = set(important_features_all).intersection(set([(2, x[1]) for x in important_features]))
intersection_2

{(2, 0),
 (2, 6),
 (2, 10186),
 (2, 10187),
 (2, 10188),
 (2, 10193),
 (2, 10236),
 (2, 10239),
 (2, 10286),
 (2, 10639),
 (2, 10694),
 (2, 10695),
 (2, 10699),
 (2, 10701),
 (2, 10741)}

In [50]:
len(intersection.union(intersection_1).union(intersection_2))/len(important_features_all)

0.4431818181818182

In [51]:
len(important_features)/len(important_features_all)

0.8295454545454546

In [52]:
0.443/0.829

0.5343787696019301

## Random Forest using only the important features for both the single field and the total problem

Case 1: single_simulation

In [53]:
X1 = np.concatenate([slices[i, :, j].reshape((-1, 1)) for (i, j) in important_features_all], axis=1)

In [54]:
X2 = np.concatenate([slices[i, :, j].reshape((-1, 1)) for (i, j) in important_features], axis=1)

In [57]:
print(X1.shape, X2.shape, y1.shape, y2.shape)

(7680, 88) (7680, 73) (7680,) (7680,)


In [58]:
X1_train, X1_test, X2_train, X2_test, y1_train, y1_test, y2_train, y2_test = \
    train_test_split(X1, X2, y1, y2, test_size=0.2)

In [84]:
model_11 = RandomForestRegressor(n_estimators=100)
model_12 = RandomForestRegressor(n_estimators=100)
model_21 = RandomForestRegressor(n_estimators=100)
model_22 = RandomForestRegressor(n_estimators=100)

In [85]:
model_11.fit(X1_train, y1_train)
model_12.fit(X1_train, y2_train)
model_21.fit(X2_train, y1_train)
model_22.fit(X2_train, y2_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [86]:
score_11 = mean_absolute_error(y1_test, model_11.predict(X1_test))
score_12 = mean_absolute_error(y2_test, model_12.predict(X1_test))
score_21 = mean_absolute_error(y1_test, model_21.predict(X2_test))
score_22 = mean_absolute_error(y2_test, model_22.predict(X2_test))

In [87]:
print(score_11, score_12, score_21, score_22)

0.000384319467149441 0.0007430684626557733 0.0004889102142923754 0.0010202162621454523


In [89]:
print(np.mean(y1), y2.mean())

0.03073030972208013 0.08123853839464416


Case 2: use another simulation as test

In [63]:
sim_code_test = 'sf'
slices_test =  read_pickle(path+f'slices_top_features_end_{sim_code_test}.pickle')

In [64]:
model_11 = RandomForestRegressor(n_estimators=100)
model_12 = RandomForestRegressor(n_estimators=100)
model_21 = RandomForestRegressor(n_estimators=100)
model_22 = RandomForestRegressor(n_estimators=100)

In [65]:
model_11.fit(X1, y1)
model_12.fit(X1, y2)
model_21.fit(X2, y1)
model_22.fit(X2, y2)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [77]:
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code_test}.mat')
y_new = x['k_quantities']

In [68]:
X1_new = np.concatenate([slices_test[i, :, j].reshape(-1, 1) for i, j in important_features_all], axis=1)
X2_new = np.concatenate([slices_test[i, :, j].reshape(-1, 1) for i, j in important_features], axis=1)

In [78]:
y_new = np.concatenate([y_new[i, :, :] for i in range(y_new.shape[0])])
y_new = y_new[:len(X1_new)]
y1_new = y_new[:, 0]
y2_new = y_new[:, 1]

In [79]:
y1_new.shape

(7680,)

In [82]:
score_11 = mean_absolute_error(y1_new, model_11.predict(X1_new))
score_12 = mean_absolute_error(y2_new, model_12.predict(X1_new))
score_21 = mean_absolute_error(y1_new, model_21.predict(X2_new))
score_22 = mean_absolute_error(y2_new, model_22.predict(X2_new))

In [83]:
print(score_11, score_12, score_21, score_22)

0.009043744755882745 0.0038099001060633493 0.009355945201310075 0.00471834558049169


In [90]:
print(np.mean(y1_new), y2_new.mean())

0.039276038699720475 0.08282099771353216
